<a href="https://colab.research.google.com/github/Nomad653/Python/blob/main/Pipelines/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [391]:
import numpy as np

In [392]:
import pandas as pd

In [393]:
from sklearn.linear_model import LogisticRegression

In [394]:
from sklearn.preprocessing import StandardScaler

In [395]:
from sklearn.pipeline import Pipeline

In [396]:
from sklearn.impute import SimpleImputer

In [397]:
from sklearn.preprocessing import OneHotEncoder

In [398]:
from sklearn.compose import ColumnTransformer

In [399]:
from sklearn.ensemble import RandomForestClassifier

In [400]:
from sklearn.model_selection import train_test_split

In [401]:
from scipy.stats import zscore

In [402]:
data = pd.read_csv("/content/train.csv")

In [403]:
new_data = data.drop(columns=["HomePlanet","Name","PassengerId","Cabin"])

In [404]:
new_data['Total_billed'] = new_data.RoomService + new_data.FoodCourt + new_data.ShoppingMall +new_data.Spa +new_data.VRDeck

In [405]:
conditions = [
              (new_data['Age']<2),
              (new_data['Age']<=15), #Child
              (new_data['Age']>15) & (new_data['Age']<=18), #Teenager
              (new_data['Age']>18) & (new_data['Age']<=35),#Young adult
              (new_data['Age']>35) & (new_data['Age']<=50), #Mid-age
              (new_data['Age']>50) & (new_data['Age']<=60),#Old
              (new_data['Age']>60)


]
values = ['Infant','Child','Teenager','Young adult','Mid-age','Old','Elderly']

In [406]:
new_data['Age_group'] = np.select(conditions,values)

In [407]:
new_data

CryoSleep    Destination   Age    VIP  RoomService  FoodCourt  \
0        False    TRAPPIST-1e  39.0  False          0.0        0.0   
1        False    TRAPPIST-1e  24.0  False        109.0        9.0   
2        False    TRAPPIST-1e  58.0   True         43.0     3576.0   
3        False    TRAPPIST-1e  33.0  False          0.0     1283.0   
4        False    TRAPPIST-1e  16.0  False        303.0       70.0   
...        ...            ...   ...    ...          ...        ...   
8688     False    55 Cancri e  41.0   True          0.0     6819.0   
8689      True  PSO J318.5-22  18.0  False          0.0        0.0   
8690     False    TRAPPIST-1e  26.0  False          0.0        0.0   
8691     False    55 Cancri e  32.0  False          0.0     1049.0   
8692     False    TRAPPIST-1e  44.0  False        126.0     4688.0   

      ShoppingMall     Spa  VRDeck  Transported  Total_billed    Age_group  
0              0.0     0.0     0.0        False           0.0      Mid-age  
1             25.0   549.0    44.0         True         736.0  Young adult  
2              0.0  6715.0    49.0        False       10383.0          Old  
3            371.0  3329.0   193.0        False        5176.0  Young adult  
4            151.0   565.0     2.0         True        1091.0     Teenager  
...            ...     ...     ...          ...           ...          ...  
8688           0.0  1643.0    74.0        False        8536.0      Mid-age  
8689           0.0     0.0     0.0        False           0.0     Teenager  
8690        1872.0     1.0     0.0         True        1873.0  Young adult  
8691           0.0   353.0  3235.0        False        4637.0  Young adult  
8692           0.0     0.0    12.0         True        4826.0      Mid-age  

[8693 rows x 12 columns]

In [408]:
X  =new_data.drop("Transported",axis=1)

y = new_data["Transported"]

In [409]:
num=X.select_dtypes(include="number").columns
cat=X.select_dtypes(include="object").columns

In [410]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [411]:
num_pipeline = Pipeline(steps=[("num_imputer", SimpleImputer(strategy="most_frequent",missing_values=np.nan)),
                               ("scaler",StandardScaler())
                        ])

cat_pipeline = Pipeline(steps=[
                               ('imputer',SimpleImputer(strategy="most_frequent",missing_values=np.nan)),
                               ('onehot',OneHotEncoder(handle_unknown="ignore"))
])

In [412]:
preprocessor = ColumnTransformer([("numerical_pipeline",num_pipeline,num),
                                  ("categorical_pipeline",cat_pipeline,cat)
                                  ])

In [413]:
pipe = Pipeline(steps=[
                       ("preprocessor",preprocessor),
                       
                       ("lr",LogisticRegression(max_iter=300)),
                       

])

In [414]:
X_train = pipe.fit(X_train,y_train)

In [415]:
predictions = pipe.predict(X_test)

In [416]:
from sklearn.metrics import accuracy_score

In [417]:
from sklearn.model_selection import GridSearchCV

In [418]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [419]:
acc1 = accuracy_score(y_test,predictions)

In [420]:
acc1

0.7763082231167338